In [1]:
import numpy as np
from numpy.polynomial import Polynomial
import matplotlib.pyplot as plt
import time
from numba import njit
# %matplotlib qt

## Setup



In [2]:
V=100
L=100
eps=1e-5

In [99]:
import numpy as np

V=100
L=100
eps=1e-5

def get_grid():
    return np.zeros((2*L,2*L))

def set_boundry(grid):
    grid[L, L-1:] = np.linspace(V, 0, L+1)
    grid[0:L+1,L-1] = np.linspace(0, V, L+1)
    grid[0:L,L:] = np.zeros((L,L))

def compute_grid():
    diff = eps+1

    grid = get_grid()
    last_grid = get_grid()
    set_boundry(last_grid)

    while (diff>eps):
        N+=1
        grid = get_grid()
        grid[1:-1,1:-1] = 0.25*(last_grid[1:-1, 2:] + last_grid[1:-1, :-2]+ last_grid[2:, 1:-1] + last_grid[:-2, 1:-1])
        set_boundry(grid)

        diff = np.fabs(grid-last_grid).max()
        last_grid = grid

    return grid

In [107]:
start = time.time()
grid1 = compute_grid()
print(time.time() - start)
# plt.imshow(grid1)

16813
3.1525352001190186


1682321597.734146

In [14]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X = np.arange(2*L)
Y = np.arange(L)[::-1]
X, Y = np.meshgrid(X, Y)
# Z = grid + np.rot90(grid.T,2)
# Z = grid1+grid2
Z = grid
ax.contour(X, Y, Z, 50)

In [57]:
grid2 = grid-grid1
grid2[0:L,L:] = np.zeros((L,L))
grid2.min()

0.0

In [66]:
a = np.ones(2)
np.concatenate((a,np.ones(5)))

array([1., 1., 1., 1., 1., 1., 1.])

In [94]:
# def transform()

N = 10
D = 100

x = np.array([])
y = np.array([])

for i in np.linspace(0, 0.5, N):
    x = np.concatenate((x, np.linspace(i,1, D)))
    y = np.concatenate((y, np.ones(D)*i))
    y = np.concatenate((y, np.linspace(i,1, D)))
    x = np.concatenate((x, np.ones(D)*i))

for i in range(len(x)):
    xi = x[i]
    yi = y[i]

    v = yi/xi

    if 1 >= v >= 0.5:
        x[i] = 2*v*xi
        y[i] = 2*v*yi
    elif 1 < v <= 2:
        x[i] = 2*xi/v
        y[i] = 2*yi/v

fig, ax = plt.subplots()
ax.plot(x, y, 'o', color='lightgrey')



/var/folders/jn/78bt2nn54tl2wjjmmmcq84k40000gn/T/ipykernel_93943/3665491763.py:19: RuntimeWarning: invalid value encountered in double_scalars
  v = yi/xi
/var/folders/jn/78bt2nn54tl2wjjmmmcq84k40000gn/T/ipykernel_93943/3665491763.py:19: RuntimeWarning: divide by zero encountered in double_scalars
  v = yi/xi


In [18]:
def optimized_boundry(grid):
    grid[0, L-1:] = np.linspace(V, 0, L+1)
    grid[0, :L-1] = grid[:0:-1, L-1]

def optimized_compute():
    errors = []
    N=0
    err = eps+1

    grid = np.zeros((L, 2*L))
    last_grid = np.zeros((L, 2*L))
    optimized_boundry(last_grid)

    while (err>eps):
        N+=1
        grid = optimized_compute_step(last_grid)
        optimized_boundry(grid)

        err = (grid-last_grid).max()
        errors.append(err)
        last_grid = grid

    print(N)
    return grid, N, errors

def optimized_compute_step(grid):
    new_grid = np.zeros((L, 2*L))
    new_grid[1:-1,1:-1] = 0.25*(grid[2:,2:]+grid[2:,:-2]+grid[:-2,2:]+grid[:-2,:-2])
    return new_grid

In [22]:
start = time.time()
grid, n, err = optimized_compute()
print(time.time() - start)
# plt.imshow(grid)

plt.plot(np.arange(n), np.log(err))

16813
2.962104082107544


In [3]:
@njit("f8[:,:]()", nogil=True)
def numba_compute():
    L = 100
    V = 100
    eps=1e-5
    err=eps+1

    i = np.array([0,1])

    grids = [np.zeros((L, 2*L)), np.zeros((L, 2*L))]
    grids[i[0]][0, L-1:] = np.linspace(V, 0, L+1)
    grids[i[0]][0, :L-1] = grids[i[0]][:0:-1, L-1]

    while (err>eps):

        i = i[::-1]

        # N+=1

        grids[i[0]][1:-1,1:-1] = 0.25*(grids[i[1]][2:,2:]+grids[i[1]][2:,:-2]+grids[i[1]][:-2,2:]+grids[i[1]][:-2,:-2])
        grids[i[0]][0, L-1:] = np.linspace(V, 0, L+1)
        grids[i[0]][0, :L-1] = grids[i[0]][:0:-1, L-1]

        err = (grids[i[0]]-grids[i[1]]).max()

    # print(N)
    return grids[i[0]]

In [5]:
start = time.time()
numba_grid = numba_compute()
print(time.time() - start)
# plt.imshow(numba_grid)

1.4044268131256104


In [31]:
x = np.sqrt(np.arange(L)+1)
y = grid[0,:L]
p = Polynomial.fit(x, y, 2)

fig, ax = plt.subplots()
ax.plot(x, y)
ax.plot(x, p.)

TypeError: 'Polynomial' object is not subscriptable

In [13]:
import dolfin as df

# Define the mesh
mesh = df.UnitSquareMesh(50, 50)

# Define the function space
V = df.FunctionSpace(mesh, 'P', 1)

# Define the boundary condition
u_D = df.Constant(0.0)
bc = df.DirichletBC(V, u_D, 'on_boundary')

# Define the variational problem
u = df.TrialFunction(V)
v = df.TestFunction(V)
f = df.Constant(1.0)
a = df.inner(df.grad(u), df.grad(v)) * df.dx
L = f * v * df.dx

# Compute the solution
u = df.Function(V)
df.solve(a == L, u, bc)

# Plot the solution
df.plot(u)
df.interactive()

ModuleNotFoundError: No module named 'dolfin'

In [23]:
def iterative_compute():
    N=0
    err = eps+1
    errors = []

    grid = np.zeros((L, 2*L))    
    grid[0, L-1:] = np.linspace(V, 0, L+1)

    while (err>eps):
        N+=1
        new_grid = np.zeros((L, 2*L))
        new_grid[1:-1,1:-1] = 0.25*(grid[2:,2:]+grid[2:,:-2]+grid[:-2,2:]+grid[:-2,:-2])
        new_grid[0, :L-1] = new_grid[:0:-1, L-1]

        err = (new_grid-grid).max()
        errors.append(err)
        grid = new_grid

    print(N)
    print(err)
    return grid, N, errors

start = time.time()
grid, n, err = iterative_compute()
print(time.time() - start)

plt.plot(np.arange(n), np.log(err))

16813
9.998789574359521e-06
1.7985341548919678
